# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
# --- store the filepath in a variable ---
file = "../output_data/cities.csv"

# --- read the csv file and create a dataframe ---
cities_data = pd.DataFrame(pd.read_csv(file, index_col="City ID"))

# --- display the dataframe ---
cities_data

,City,Country,Date,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
City ID,,,,,,,,,
0,Ushuaia,AR,2022-12-28 04:00:47,-54.8000,-68.3000,40.66,75,75,26.46
1,Rikitea,PF,2022-12-28 04:00:47,-23.1203,-134.9692,78.22,73,24,10.27
2,Butaritari,KI,2022-12-28 04:00:48,3.0707,172.7902,80.04,82,67,15.66
3,San Patricio,US,2022-12-28 04:00:48,28.0170,-97.5169,46.08,79,30,5.70
4,Chapais,CA,2022-12-28 03:55:49,49.7834,-74.8492,-9.62,100,65,3.67
...,...,...,...,...,...,...,...,...,...
580,Puyo,EC,2022-12-28 04:00:47,-1.4667,-77.9833,64.27,82,25,3.87
581,Juruti,BR,2022-12-28 04:03:55,-2.1522,-56.0922,72.61,97,100,2.33
582,Ilula,TZ,2022-12-28 04:03:56,-7.6766,36.0366,61.47,95,94,1.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
#access maps with unique API key
gmaps.configure(api_key=g_key)

#use Lat and Lng as locations and Humidity as the weight
locations = cities_data[["Lat", "Lng"]]
weights = cities_data["Humidity (%)"]

#customize the size of the figure and add heatmap layer to map
figure_layout = {
    'width': "100%",
    'height': '400px',
#    'border': '1px solid black',
    'padding': '1px',
#    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, center=(20.0, 5.0), zoom_level=1.9)
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights, dissipating=False)
fig.add_layer(heatmap_layer)

#set maximum intensity and point_radius settings
heatmap_layer.max_intensity = max(weights)
heatmap_layer.point_radius = 3

#display heatmap
fig

Figure(layout=FigureLayout(height='400px', padding='1px', width='100%'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
special_cities = cities_data.loc[(cities_data["Max Temp (F)"] > 60) & 
                               (cities_data["Max Temp (F)"] < 80) & 
                               (cities_data["Wind Speed (mph)"] < 10) & 
                               (cities_data["Cloudiness (%)"] == 0) &
                               (cities_data["Humidity (%)"] < 50), :].reset_index(drop=True).dropna()
special_cities

,City,Country,Date,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,Ponta do Sol,PT,2022-12-28 04:01:12,32.6667,-17.1000,63.01,27,0,5.68
1,Busselton,AU,2022-12-28 04:00:26,-33.6500,115.3333,77.25,28,0,1.21
2,Rāmgarh,IN,2022-12-28 03:57:43,23.6333,85.5167,62.65,22,0,6.62
3,Tura,IN,2022-12-28 04:01:46,25.5198,90.2201,67.12,37,0,2.13
4,Ingrāj Bāzār,IN,2022-12-28 04:02:48,25.0000,88.1500,65.71,28,0,7.87
5,Rafaela,AR,2022-12-28 03:58:42,-31.2503,-61.4867,71.98,36,0,1.01
6,Gangāpur,IN,2022-12-28 03:58:51,26.4833,76.7167,62.71,24,0,3.09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
hotel_df = special_cities[["City", "Country", "Lat", "Lng"]].copy()

# --- Add a "Hotel Name" column to the DataFrame with null values ---
hotel_df["Hotel Name"] = np.nan
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Ponta do Sol,PT,32.6667,-17.1000,NaN
1,Busselton,AU,-33.6500,115.3333,NaN
2,Rāmgarh,IN,23.6333,85.5167,NaN
3,Tura,IN,25.5198,90.2201,NaN
4,Ingrāj Bāzār,IN,25.0000,88.1500,NaN
5,Rafaela,AR,-31.2503,-61.4867,NaN
6,Gangāpur,IN,26.4833,76.7167,NaN


In [12]:
#set up parameters for querying Google Places API
target_search = "hotel"
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

#define base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#use a for loop to go through each city in the dataframe and make an API call

for index, row in hotel_df.iterrows():
    
    #get the coordinates and store in params dictionary
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    #call the API and get hotel details for each city
    hotel_data = requests.get(base_url, params).json()
    
    #use exception handling to save the first hotel name for each city
    
    try:
        hotel_name = hotel_data["results"][0]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
        print(f"City '{row['City']}' found! The nearest hotel is {hotel_name}")
        print("-------------------------------------------------------------------------")
              
    except:
        print(f"Could not find information for city '{row['City']}'... skipping.")    
        print("-------------------------------------------------------------------------")
        

City 'Ponta do Sol' found! The nearest hotel is Hotel do Campo
-------------------------------------------------------------------------
City 'Busselton' found! The nearest hotel is Presidential Bungalow
-------------------------------------------------------------------------
City 'Rāmgarh' found! The nearest hotel is Flagship 45144 Hotel Annie
-------------------------------------------------------------------------
City 'Tura' found! The nearest hotel is Hotel Polo Orchid
-------------------------------------------------------------------------
City 'Ingrāj Bāzār' found! The nearest hotel is Continental Lodge
-------------------------------------------------------------------------
City 'Rafaela' found! The nearest hotel is Amérian Rafaela Hotel
-------------------------------------------------------------------------
City 'Gangāpur' found! The nearest hotel is Hotel Aagman - Best Family Rooms | Hotels in Gangapur Rajasthan
-----------------------------------------------------------

In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
#add marker layer on top of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='400px', padding='1px', width='100%'))